<a href="https://colab.research.google.com/github/ashishjoshi12/Hacakthon2025/blob/main/project_claude.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install langchain-community llama-cpp-python pydantic==1.10.8 email extract-msg PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 MB 9.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.4/146.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.3 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [1]:
pip install langchain-community

  Using cached pydantic-2.10.6-py3-none-any.whl.metadata (30 kB)
Using cached pydantic-2.10.6-py3-none-any.whl (431 kB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.8
    Uninstalling pydantic-1.10.8:
      Successfully uninstalled pydantic-1.10.8


In [4]:
pip install llama-cpp-python

  Using cached llama_cpp_python-0.3.8.tar.gz (67.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.5 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.8-cp311-cp311-linux_x86_64.whl size=5959650 sha256=598fb33beaf51c84f8b3a28aaf24cd1bc001a770ff619d6455356365b45bafe6
  Stored in directory: /root/.cache/pip/wheels/c0/03/66/eb3810eafd55d921b2be32896d1f44313996982360663aa80b
Successfully built llama-cpp-python


In [7]:
pip install pydantic==2.7.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.4/109.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.0/409.0 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 42.5 MB/s eta 0:00:00
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.27.2
    Uninstalling pydantic_core-2.27.2:
      Successfully uninstalled pydantic_core-2.27.2
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.8
    Uninstalling pydantic-1.10.8:
      Successfully uninstalled pydantic-1.10.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 2.0.5 requires pydantic>=2.9.2, but you have pydantic 2.7.4 which is incompatible.


In [3]:
pip install PyPDF2

In [7]:
pip install email

  Using cached email-4.0.2.tar.gz (1.2 MB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [1]:
# filepath: c:\Users\ajosh\OneDrive\Desktop\MachineLearning\Hackathon2025\loan_classifier.py
from langchain_community.llms import LlamaCpp
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from pathlib import Path
import json
import PyPDF2
from google.colab import files
import io

class ServiceRequest(BaseModel):
    request_type: str = Field(description="Main category of the request")
    sub_request: str = Field(description="Specific sub-category of the request")
    confidence_score: float = Field(description="Confidence score of the classification")
    supporting_text: str = Field(description="Relevant text that supports this classification")

In [2]:
class LoanServiceRequestClassifier:
    def __init__(self, model_path: str = "llama-3-7b.gguf"):
        self.llm = LlamaCpp(
            model_path=model_path,
            temperature=0.1,
            max_tokens=2000,
            n_gpu_layers=40,
            n_ctx=4096,
            top_p=0.95,
            verbose=True
        )

        self.output_parser = PydanticOutputParser(pydantic_object=ServiceRequest)
        self._setup_prompt()

    def _setup_prompt(self):
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a loan servicing expert. Analyze the provided text and classify it into appropriate request and sub-request categories.

            Request Types and Sub-categories:
            1. Loan Modification
               - Payment Reduction
               - Term Extension
               - Interest Rate Adjustment
               - Principal Forbearance
               - Hardship Assistance

            2. Payment Processing
               - Payment Setup/Change
               - Auto-Pay Enrollment
               - Payment Dispute
               - Late Payment Resolution
               - Payment Application

            3. Documentation
               - Statement Request
               - Payment History
               - Tax Documents (1098)
               - Loan Agreement Copy
               - Insurance Documents

            4. Account Updates
               - Contact Information
               - Insurance Updates
               - Banking Details
               - Name/Ownership Change
               - Death/Estate Processing

            5. Payoff/Discharge
               - Payoff Statement
               - Early Payoff Quote
               - Lien Release
               - Satisfaction Letter
               - Discharge Processing

            Analyze the content and output a JSON object with:
            - request_type: Main category
            - sub_request: Specific sub-category
            - confidence_score: 0.0 to 1.0
            - supporting_text: Relevant evidence"""),
            ("human", "{content}")
        ])

    def classify_request(self, email_content: str, attachment_text: str = "") -> ServiceRequest:
        full_content = f"Email:\n{email_content}\n\nAttachment:\n{attachment_text}"
        chain = self.prompt | self.llm | self.output_parser
        return chain.invoke({"content": full_content})

In [3]:
def read_pdf_content(pdf_file):
    try:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text() + "\n"
        return text
    except Exception as e:
        return f"Error reading PDF: {str(e)}"

def process_uploaded_files():
    print("Upload your email file (EML/MSG) and any attachments...")
    uploaded = files.upload()

    email_content = ""
    attachment_content = ""

    for filename, content in uploaded.items():
        if filename.lower().endswith(('.eml', '.msg')):
            email_content = content.decode('utf-8')
        elif filename.lower().endswith('.pdf'):
            attachment_content = read_pdf_content(io.BytesIO(content))

    return email_content, attachment_content

In [4]:
def main():
    # Download Llama 3 model (you'll need to modify this based on your setup)
    MODEL_PATH = "llama-3-7b.gguf"

    # Process uploaded files
    email_content, attachment_content = process_uploaded_files()

    # Initialize classifier
    classifier = LoanServiceRequestClassifier(model_path=MODEL_PATH)

    # Get classification
    try:
        classification = classifier.classify_request(
            email_content=email_content,
            attachment_text=attachment_content
        )

        print("\nClassification Results:")
        print(f"Request Type: {classification.request_type}")
        print(f"Sub-Request: {classification.sub_request}")
        print(f"Confidence Score: {classification.confidence_score:.2f}")
        print(f"\nSupporting Text:\n{classification.supporting_text}")

        # Save results as JSON
        results = classification.dict()
        with open('classification_results.json', 'w') as f:
            json.dump(results, f, indent=2)

        # Download results
        files.download('classification_results.json')

    except Exception as e:
        print(f"Error during classification: {str(e)}")

if __name__ == "__main__":
    main()

Upload your email file (EML/MSG) and any attachments...


Saving fin_payoff_letter.pdf to fin_payoff_letter (1).pdf


ValidationError: 1 validation error for LlamaCpp
  Value error, Could not load Llama model from path: llama-3-7b.gguf. Received error Model path does not exist: llama-3-7b.gguf [type=value_error, input_value={'model_path': 'llama-3-7...: None, 'grammar': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.7/v/value_error